In [1]:
import binascii
import sys
import traceback
from block import Block
from helper import read_varint, hash256
from tx import Tx
import subprocess
import os
from time import sleep
from subprocess import Popen, PIPE
import logging
#logging.basicConfig(filename='parsing.log',level=logging.DEBUG,format='%(asctime)s %(message)s')

In [2]:
from neo4j import GraphDatabase

In [3]:
from logging.handlers import RotatingFileHandler

log_formatter = logging.Formatter('%(asctime)s %(levelname)s %(funcName)s(%(lineno)d) %(message)s')

logFile = 'log/parsing.log'

my_handler = RotatingFileHandler(logFile, mode='a', maxBytes=10*1024*1024,backupCount=6, encoding=None, delay=0)
my_handler.setFormatter(log_formatter)
my_handler.setLevel(logging.INFO)

app_log = logging.getLogger('root')
app_log.setLevel(logging.INFO)

app_log.addHandler(my_handler)

In [17]:
class BlockChainDB(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password), encrypted=False)#REVISE LATER FOR ENCRYPTION!!!

    def close(self):
        self._driver.close()
        
        
    @staticmethod
    def _new_address(tx, address,i,change_addr):
        if change_addr: addr_type = "change"
        else: addr_type = "recipient"
        result = tx.run("CREATE a = (:address {address:$address, acc_index:$index, type:$kind, created:timestamp()}) "
                        "RETURN a ", address=address, index = i, kind = addr_type)
        return result.single()
    
    
    @staticmethod
    def _new_output(tx,amount, script_pubkey,tx_id,out_index):
        result = tx.run("MATCH (n:transaction {id : $tx_id}) "
                        "CREATE p = (:output {out_index:$out_index, script_pubkey:$script_pubkey})<-[:CREATES]-(n) "
                        "RETURN p ", tx_id = tx_id, script_pubkey=script_pubkey, out_index=out_index, local_index=local_index)
        return result.single()
    
    @staticmethod
    def _spend_outputs(tx):
        new_txs = tx.run("MATCH (tx:transaction) WHERE NOT (tx)-[:SPENDS]->() RETURN tx.id, tx.inputs")
        print(new_txs.data())
        for tx in new_txs.data():
            for tx_input in tx["tx.inputs"]:
                tx_in = tx.input.split(":")
                prev_tx=tx_in[0]
                index = tx_in[1]
                if index == 4294967295 or prev_tx == (b'\x00'*32).hex():
                    app_log.info("Coinbase Transaction " + tx['tx.id'])
                                 
                else:
                    #app_log(f"input index = {tx_in.prev_index}, input prev tx = {tx_in.prev_tx}")
                    result = tx.run("MATCH (t:transaction {id:$tx_id}), (o:output {index:$index})<-[:CREATES]-(:transaction {id:$prev_tx}) "
                                "CREATE (t)-[r:SPENDS]->(o)"
                                "RETURN r",
                                tx_id=tx['tx.id'], index=index, prev_tx=prev_tx)
                    if result.single(): app_log.info(f"connected input: {index}")
                    else: app_log.error(f"Failed at connecting input: {index}")
                                 
        return True
        
    
    @staticmethod
    def _new_tx(tx,block_id, version, locktime, tx_id, _inputs, outputs, segwit, i):
        
        inputs = [str(tx_in) for tx_in in _inputs]
        print(inputs)
                                 
        result = tx.run("MATCH (b:block {id:$block_id}) "
                        "MERGE (t:transaction {id:$tx_id}) "
                        "SET t.version=$version, t.segwit=$segwit, t.locktime=$locktime "
                        "MERGE (t)<-[:CONTAINS {i:$i}]-(b) "
                        "WITH t "
                        "FOREACH (input in $_inputs | "
                        "MERGE (in :output {index:input.prev_index})<-[:CREATES]-(:transaction {id:$input.prev_tx}) "
                        "MERGE (in)-[:SPENDS {script_sig:$input.script_sig, witness:$input.witness}]-(t) "
                        "RETURN t ",
                        tx_id=tx_id, segwit=segwit, version=version, locktime=locktime, block_id=block_id,i=i)
        
        if result.single(): app_log.info(f"created tx: {tx_id}\n{result.single()}")
        else: app_log.error(f"FAILED AT CREATING TX {tx_id}")
        
        for index,tx_out in enumerate(outputs):
            result = tx.run("MATCH (t:transaction {id : $tx_id}) "
                        "MERGE (o:output {index:$index})<-[:CREATES]-(t) "
                        "SET o.amount=$amount, o.script_pubkey:$script_pubkey "
                        "RETURN o ", 
                        tx_id = tx_id,amount=tx_out.amount,script_pubkey=str(tx_out.script_pubkey), index=index)
            if result.single(): app_log.info(f"created output: {tx_out.amount}:{index}")
            else: app_log.error(f"FAILED AT CREATING OUTPUT {index}")  
        
        return True
    
    @staticmethod
    def _link_blocks(tx):
        new_blocks = tx.run("MATCH (blk:block) WHERE NOT (blk)<-[:LINKS]-() RETURN blk.id, blk.prev_block")
        #print(new_blocks.data()[-10:]) #IF I PRINT HERE, IT DOESN'T WORK 
        #print("starting to link blocks...")
        for block in new_blocks.data():
            
            result = tx.run("MATCH (a:block {id:$block_id}) MATCH (b:block {id:$prev_block})"
                            "CREATE p = (a)<-[r:LINKS]-(b) RETURN p", 
                            block_id=block["blk.id"], prev_block=block["blk.prev_block"])
            if result.data() is not None: app_log.info(result.data())
            else : app_log.error(block)
        return True
    
    @staticmethod
    def _new_block(tx,block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx):
        #_height = tx.run( "MATCH (u:block) RETURN COUNT (u) ").single()[0] 
        #pblock = tx.run("MATCH (prev_block:block {id:$prev_block}) RETURN prev_block", prev_block=prev_block)
        #if not pblock.single():
         #   print(f"COULD NOT FIND PREVIOUS BLOCK {prev_block}.")
        result = tx.run("MERGE (block:block {id:$block_id})"
                        "SET block.n_tx=$n_tx,block.nonce=$nonce,block.merkle_root:$merkle_root,block.bits=$bits,block.timestamp:$timestamp,block.version=$version"
                        "MERGE (prevblock:block {id:$prev_block}) "
                        "MERGE (block)<-[:chain]-(prevblock)"
                        "RETURN block",
                        block_id=block_id, version=version, prev_block=prev_block, merkle_root=merkle_root, timestamp=timestamp, 
                        bits=bits, nonce=nonce, n_tx=n_tx)
        return result.single()

    def new_address(self, address,i,change_addr):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_address, address,i,change_addr)
            print(result)
            
    def new_output(self, address,tx_id,out_index):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_utxo, address,tx_id,out_index)
            print(result)
                                 
    def spend_outputs(self):
        with self._driver.session() as session:
            result = session.write_transaction(self._spend_outputs)
            
    def new_tx(self, block_id, version, locktime, tx_id, inputs, outputs, segwit):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_tx, block_id, version, locktime, tx_id, inputs, outputs, segwit)
            
    def link_blocks(self):
        with self._driver.session() as session:
            result = session.write_transaction(self._link_blocks)
                  
    def new_block(self,block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx):
        with self._driver.session() as session:
            result = session.write_transaction(self._new_block,block_id,version, prev_block,merkle_root,timestamp,bits,nonce,n_tx)
            if result: app_log.info(f"CREATED BLOCK {block_id}")
            else: app_log.error(f"FAILED AT CREATING BLOCK {block_id}")
            #print(result)
  
      

In [18]:
db = BlockChainDB( "neo4j://localhost:7687" ,"neo4j" ,"wallet" )

In [12]:
#f = open("block_output.txt", "w")
#ok = True
#counter = 0
def parse_blockchain():
    for file in range(2040)
    with open(f"blocks_demo/blk{file:05}.dat","rb") as block_file:
        #for block_in_file in range(1000,1200):
        while ok:
            #try:
            this_block = Block.parse_from_blk(block_file)
            f.write(f"Block: \nversion:{this_block.version}\nPrevious block: {this_block.prev_block}\nMerkle: {this_block.merkle_root}")
            f.write(f"\nTimeStamp:{this_block.timestamp}\nNonce block: {this_block.nonce}\nBits: {this_block.bits}")
            f.write(f"\nNumber of Transactions: {this_block.tx_hashes}\n\nTransactions:")
            #for transaction in range(countOfTransactions):

            header = this_block.version.to_bytes(4,"little")+this_block.prev_block[::-1]+this_block.merkle_root[::-1]+this_block.timestamp.to_bytes(4,"little") + this_block.bits + this_block.nonce

            block_id = hash256(header)[::-1]
            #block_id = hex(int.from_bytes(block_id,"big"))[2:]
            #print(int.from_bytes(this_block.nonce,"big"))
            db.new_block(block_id.hex(),this_block.version, this_block.prev_block.hex(), 
                         this_block.merkle_root.hex(),this_block.timestamp, 
                         int.from_bytes(this_block.bits,"big"), 
                         int.from_bytes(this_block.nonce,"big"),
                         this_block.tx_hashes)

            for transaction in range(this_block.tx_hashes):
                tx = Tx.parse(block_file)
                db.new_tx(block_id.hex(), tx.version, tx.locktime, tx.id(), tx.tx_ins, tx.tx_outs, tx.segwit)
            """
            except Exception as e:
                print(e.with_traceback)
                break

            counter+=1
            if counter >=1000: ok=False
            """



    f.close()    
    
 

SyntaxError: invalid syntax (<ipython-input-12-05cb170e0d66>, line 5)

In [19]:
db.link_blocks()

KeyboardInterrupt: 

In [32]:
os.chdir("/Users/oscareduardosernarosero/Desktop/neo4j-community-4.0.3/bin")
whereami = subprocess.check_output("pwd")
print(whereami)
if whereami == b"/Users/oscareduardosernarosero/Desktop/neo4j-community-4.0.3/bin\n": print("ok")
print(subprocess.check_output("ls"))
start_neo4j = subprocess.check_output("./neo4j start",shell = True)
if "Started neo4j" in str(start_neo4j): print("Sarted Neo4j successfully...")
else: print("Failed starting Neo4j.")
sleep(2)

In [57]:
#os.chdir("/Users/oscareduardosernarosero/Desktop/neo4j-community-4.0.3/bin")
#output = Popen(["./cypher-shell","-u","neo4j","-p","wallet"],stdout=PIPE, stderr=PIPE)
subprocess.call("./cypher-shell -u neo4j -p wallet",shell = True)
output = subprocess.check_output("MATCH (n) RETURN n",shell = True)
print(output)

CalledProcessError: Command 'MATCH (n) RETURN n' returned non-zero exit status 2.

In [54]:
stdout, stderr = output.communicate("\tMATCH (n) RETURN n\n")
print (stdout)
#subprocess.call("neo4j",shell = True)
#sleep(2)
#loggedin = subprocess.check_output("wallet",shell = True)
#if "Connected to Neo4j 4.0.3 at neo4j://localhost:7687 as user neo4j" in str(loggedin):print("connected")

b''


In [47]:
with open("blocks_demo/blk00000.dat","rb") as block_file:
    for line in range(10):
        charlist =[]
        for ch in range(100):
            char=""
            try:
                #print(binascii.hexlify(block_file.read(60)))
                char = block_file.read(1).decode("ascii")
                #print(type(char))
            except:
                char = binascii.hexlify(block_file.read(1)).decode("utf-8")
                #print(type(char))
                #print([( block_file.read(1).decode("ascii")). for x in range(60) ])
            charlist.append(char)
            

        print("".join(charlist))       
        #except Exception as e:
        #    excType, excValue, excTraceback = sys.exc_info()
        #    print(excType, excValue, excTraceback, file = sys.stdout)
    

bed9                                     ;ed7a{7a2c>gv611b88512:b84b^J)5fIff 2b|                        
           ffffMff EThe Times 03/Jan/2009 Chancellor on brink of second bailout for banksffff 05*   
CAgfdfeUH'ga6q010\a8(39	79beaab6Ibc?L38f3U1e125c8Mba574cp+k1d_00   bed900     o8c
f172a646634f1e6556819     20Q1eK44beh14g{a354b1b657
#>a66Iff 6201                                   ffffff ffff 05*   CAb5853Q72j,e616 1381:b|f8b94{3cR753715e00414
e6"72f62s2c23B58ac    bed900     H`181b 7e908aBuAo51Y86h9a00   fd54%1czZed48Xbbf\3674N2c1`"0fb066Iff bda                  
                 ffffff ffff 05*   CAr245bPR(c319Laa56df7b97a@@c0se690d4f8R721g3e#dF1779fc41*31kw00   bed900     ddcca3a1
081a]p0a7bb66bc_bj    D72"`d8]a9fb5f09877bb7b776|1c99]66Iff edm                                   ffffff ffff 05*
   CAb9e7l[)f995a4da7c60ad80fff19#89qa00c'&749ef9u3e5P9bo<aa00   bed900     IDF621c,ta550bo>@baf2a355

In [34]:
hex(20)

'0x14'

In [6]:

#logger = logging.getLogger('example')


with open("test_file.txt","r") as block_file:
    
    for x in range(3):
        app_log.info(block_file.read(4))
        
        
        

In [7]:
p =[{'blk.id': '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f', 'blk.prev_block': '0000000000000000000000000000000000000000000000000000000000000000'}, {'blk.id': '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048', 'blk.prev_block': '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f'}, {'blk.id': '000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd', 'blk.prev_block': '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048'}, {'blk.id': '0000000082b5015589a3fdf2d4baff403e6f0be035a5d9742c1cae6295464449', 'blk.prev_block': '000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd'}, {'blk.id': '000000004ebadb55ee9096c9a2f8880e09da59c0d68b1c228da88e48844a1485', 'blk.prev_block': '0000000082b5015589a3fdf2d4baff403e6f0be035a5d9742c1cae6295464449'}, {'blk.id': '000000009b7262315dbf071787ad3656097b892abffd1f95a1a022f896f533fc', 'blk.prev_block': '000000004ebadb55ee9096c9a2f8880e09da59c0d68b1c228da88e48844a1485'}, {'blk.id': '000000003031a0e73735690c5a1ff2a4be82553b2a12b776fbd3a215dc8f778d', 'blk.prev_block': '000000009b7262315dbf071787ad3656097b892abffd1f95a1a022f896f533fc'}, {'blk.id': '0000000071966c2b1d065fd446b1e485b2c9d9594acd2007ccbd5441cfc89444', 'blk.prev_block': '000000003031a0e73735690c5a1ff2a4be82553b2a12b776fbd3a215dc8f778d'}, {'blk.id': '00000000408c48f847aa786c2268fc3e6ec2af68e8468a34a28c61b7f1de0dc6', 'blk.prev_block': '0000000071966c2b1d065fd446b1e485b2c9d9594acd2007ccbd5441cfc89444'}, {'blk.id': '000000008d9dc510f23c2657fc4f67bea30078cc05a90eb89e84cc475c080805', 'blk.prev_block': '00000000408c48f847aa786c2268fc3e6ec2af68e8468a34a28c61b7f1de0dc6'}]

In [8]:
for r in p:
    print(r)

{'blk.id': '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f', 'blk.prev_block': '0000000000000000000000000000000000000000000000000000000000000000'}
{'blk.id': '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048', 'blk.prev_block': '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f'}
{'blk.id': '000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd', 'blk.prev_block': '00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048'}
{'blk.id': '0000000082b5015589a3fdf2d4baff403e6f0be035a5d9742c1cae6295464449', 'blk.prev_block': '000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd'}
{'blk.id': '000000004ebadb55ee9096c9a2f8880e09da59c0d68b1c228da88e48844a1485', 'blk.prev_block': '0000000082b5015589a3fdf2d4baff403e6f0be035a5d9742c1cae6295464449'}
{'blk.id': '000000009b7262315dbf071787ad3656097b892abffd1f95a1a022f896f533fc', 'blk.prev_block': '000000004ebadb55ee9096c9a2f8880e09da59c0d68b1c228da88e48844a1485'}
{'blk.id':

In [ ]:
os.chdir("block_demo")
files = subprocess.check_output("ls")
files